<a href="https://colab.research.google.com/github/NainaMKumar/Arrythmia_detection/blob/main/Arrythmia_Detection_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
pip install neurokit2

In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import neurokit2 as nk

from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE

import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.optimizers import Adam
from keras.metrics import Precision
from keras.metrics import Recall
from keras.metrics import AUC
from keras.utils import to_categorical

In [40]:
data = pd.read_csv('/content/arrhythmia.data', sep = ',')
# print(data)

In [41]:
# Deal with missing values: '?'
data.replace('?', np.nan, inplace=True)

# Replace column headers
new_column_headers = np.linspace(1, 280, 280, dtype=int)
data.columns = new_column_headers
display(data)


,1,2,3,4,5,6,7,8,9,10,...,271,272,273,274,275,276,277,278,279,280
0,56,1,165,64,81,174,401,149,39,25,...,0.0,8.5,0.0,0.0,0.0,0.2,2.1,20.4,38.8,6
1,54,0,172,95,138,163,386,185,102,96,...,0.0,9.5,-2.4,0.0,0.0,0.3,3.4,12.3,49.0,10
2,55,0,175,94,100,202,380,179,143,28,...,0.0,12.2,-2.2,0.0,0.0,0.4,2.6,34.6,61.6,1
3,75,0,190,80,88,181,360,177,103,-16,...,0.0,13.1,-3.6,0.0,0.0,-0.1,3.9,25.4,62.8,7
4,13,0,169,51,100,167,321,174,91,107,...,-0.6,12.2,-2.8,0.0,0.0,0.9,2.2,13.5,31.1,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
446,53,1,160,70,80,199,382,154,117,-37,...,0.0,4.3,-5.0,0.0,0.0,0.7,0.6,-4.4,-0.5,1
447,37,0,190,85,100,137,361,201,73,86,...,0.0,15.6,-1.6,0.0,0.0,0.4,2.4,38.0,62.4,10
448,36,0,166,68,108,176,365,194,116,-85,...,0.0,16.3,-28.6,0.0,0.0,1.5,1.0,-44.2,-33.2,2
449,32,1,155,55,93,106,386,218,63,54,...,-0.4,12.0,-0.7,0.0,0.0,0.5,2.4,25.0,46.6,1


In [42]:
print(data.iloc[:, 28:39])

     29  30  31  32  33  34  35  36  37  38  39
0    64   0   0   0  24   0   0   0   0   0   0
1    56  52   0   0  40   0   0   0   0   0   0
2    64  36   0   0  36   0   0   0   0   0   0
3    40  24   0   0  24   0   0   0   0   0   0
4    44  36   0   0  44   0   0   0   0   0   0
..   ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..
446  44  40   0   0  32   0   0   0   0   0   0
447  56   0   0   0  32   0   0   0   0   0   0
448  40  56   0   0  32   0   0   0   0   0   0
449  64   0   0   0  40   0   0   0   0   0   0
450  56   0   0   0  36   0   0   0   0   0   0

[451 rows x 11 columns]


In [43]:
#np.nan is undefined --> NaN
data = data.apply(pd.to_numeric, errors='coerce')
# display(data)

from sklearn.impute import SimpleImputer
imp = SimpleImputer(strategy="most_frequent")
data = imp.fit_transform(data)
# print(imp.fit_transform(data))

data = pd.DataFrame(data)


In [44]:
data = data[data.iloc[:, 279] != 8]

In [45]:
X = data.iloc[:, data.columns!= '280']
y = data.iloc[:, -1]

print(X)

      0    1      2     3      4      5      6      7      8      9    ...  \
0    56.0  1.0  165.0  64.0   81.0  174.0  401.0  149.0   39.0   25.0  ...   
1    54.0  0.0  172.0  95.0  138.0  163.0  386.0  185.0  102.0   96.0  ...   
2    55.0  0.0  175.0  94.0  100.0  202.0  380.0  179.0  143.0   28.0  ...   
3    75.0  0.0  190.0  80.0   88.0  181.0  360.0  177.0  103.0  -16.0  ...   
4    13.0  0.0  169.0  51.0  100.0  167.0  321.0  174.0   91.0  107.0  ...   
..    ...  ...    ...   ...    ...    ...    ...    ...    ...    ...  ...   
446  53.0  1.0  160.0  70.0   80.0  199.0  382.0  154.0  117.0  -37.0  ...   
447  37.0  0.0  190.0  85.0  100.0  137.0  361.0  201.0   73.0   86.0  ...   
448  36.0  0.0  166.0  68.0  108.0  176.0  365.0  194.0  116.0  -85.0  ...   
449  32.0  1.0  155.0  55.0   93.0  106.0  386.0  218.0   63.0   54.0  ...   
450  78.0  1.0  160.0  70.0   79.0  127.0  364.0  138.0   78.0   28.0  ...   

     270   271   272  273  274  275  276   277   278   279  
0 

In [46]:
#figure out which features are useful

estimator = LogisticRegression()
rfe = RFE(estimator, n_features_to_select=10)
rfe.fit(X, y)

selected_features = rfe.support_
print(selected_features)

feature_rankings = rfe.ranking_
print(feature_rankings)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

[False False False False False  True False False False False False False
 False False  True False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False  True  True False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False  True False False False False False False False
 False False False False False False False False Fa

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [47]:
feature_columns = []

for i in range(len(feature_rankings)):
    if feature_rankings[i] == 1:
        column_number = i + 1
        feature_columns.append(column_number)

print(feature_columns)
feature_columns = feature_columns[1:6]
print(feature_columns)

[6, 15, 112, 113, 149, 228, 238, 278, 279, 280]
[15, 112, 113, 149, 228]


In [48]:
X = np.array(X)
X = X[:, feature_columns]

print(X.shape)

(450, 5)


In [49]:
# Replace values 14, 15, and 16 with 11, 12, and 13, respectively
y.replace({14: 11, 15: 12, 16: 13}, inplace=True)
y = y - 1

print(np.unique(y))


[ 0.  1.  2.  3.  4.  5.  6.  8.  9. 10. 11. 12.]


In [50]:
# Determine the number of classes
num_classes = len(np.unique(y))
print(num_classes)

12


In [51]:
#class weights

class_weights = class_weight.compute_class_weight('balanced', classes = np.unique(y),
                                                   y = y)
class_weights_dict = dict(enumerate(class_weights))


In [52]:
print(class_weights_dict)

{0: 0.15306122448979592, 1: 0.8522727272727273, 2: 2.5, 3: 2.5, 4: 2.8846153846153846, 5: 1.5, 6: 12.5, 7: 4.166666666666667, 8: 0.75, 9: 9.375, 10: 7.5, 11: 1.7045454545454546}


In [53]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [54]:
print(X_train.shape)
print(y_train.shape)
print(len(np.unique(y_train)))

(315, 5)
(315,)
12


In [55]:
# Convert categorical labels to one-hot encoded format
y_train = tf.one_hot(y_train, num_classes)
y_test = tf.one_hot(y_test, num_classes)

# Verify the shape of y_encoded
print(y_train.shape)

(315, 12)


In [56]:
model = keras.Sequential()
model.add(layers.Dense(100, activation = 'relu'))
model.add(layers.Dense(50, activation = 'relu'))
model.add(layers.Dense(num_classes, activation = 'softmax'))


In [57]:
learning_rate = 0.001
optimizer = Adam(learning_rate=learning_rate)

In [58]:
batch_size = 32

model.compile(
    loss= 'categorical_crossentropy',
    optimizer=optimizer,
    metrics = ['accuracy', Precision(), Recall(), AUC(curve = 'ROC')]
)

history = model.fit(
    X_train,
    y_train,
    epochs= 16,
    batch_size=batch_size,
    shuffle=False,
    validation_data = (X_test, y_test),
    class_weight = class_weights_dict
)

Epoch 1/16
10/10 [==============================] - 2s 63ms/step - loss: 19.9164 - accuracy: 0.0413 - precision_1: 0.0460 - recall_1: 0.0399 - auc_1: 0.3817 - val_loss: 11.8661 - val_accuracy: 0.0741 - val_precision_1: 0.0606 - val_recall_1: 0.0315 - val_auc_1: 0.3985
Epoch 2/16
10/10 [==============================] - 0s 8ms/step - loss: 6.3124 - accuracy: 0.1206 - precision_1: 0.1276 - recall_1: 0.1030 - auc_1: 0.4353 - val_loss: 10.8470 - val_accuracy: 0.1556 - val_precision_1: 0.1639 - val_recall_1: 0.1575 - val_auc_1: 0.5026
Epoch 3/16
10/10 [==============================] - 0s 7ms/step - loss: 4.8895 - accuracy: 0.1429 - precision_1: 0.1583 - recall_1: 0.1262 - auc_1: 0.4796 - val_loss: 7.4566 - val_accuracy: 0.1778 - val_precision_1: 0.1905 - val_recall_1: 0.1575 - val_auc_1: 0.4737
Epoch 4/16
10/10 [==============================] - 0s 8ms/step - loss: 3.6077 - accuracy: 0.1556 - precision_1: 0.1739 - recall_1: 0.1329 - auc_1: 0.4801 - val_loss: 5.0776 - val_accuracy: 0.1704 -

In [59]:
loss, accuracy, precision, recall, auc = model.evaluate(X_test, y_test, batch_size = 64)

3/3 [==============================] - 0s 13ms/step - loss: 2.4261 - accuracy: 0.2370 - precision_1: 0.2574 - recall_1: 0.2047 - auc_1: 0.7867


In [60]:
!python main.py

In [61]:
import main

In [62]:
df = pd.read_csv('/content/ECGFindings.csv')
fileName = df['fileName']

In [63]:
def return_calculations(ecg_signal):

  D3_lead = ecg_signal[2]
  V1_lead = ecg_signal[6]
  V2_lead = ecg_signal[7]
  V3_lead = ecg_signal[8]
  V6_lead = ecg_signal[11]

  Q_wave_D3 = nk.ecg_process(D3_lead)['ECG_Q_Peaks'].mean()
  Q_wave_V3 = nk.ecg_process(V3_lead)['ECG_Q_Peaks'].mean()
  R_wave_V3 = nk.ecg_process(V3_lead)['ECG_R_Peaks'].mean()
  R_wave_V6 = nk.ecg_process(V6_lead)['ECG_R_Peaks'].mean()

  return Q_wave_D3, Q_wave_V3,R_wave_V3, R_wave_V6

In [64]:
# heart_rate = []
# avg_width_Q_wave_D3 = []
# avg_width_Q_wave_V3 = []
# avg_width_R_wave_V3 = []
# avg_width_R_wave_V6 = []


# for val in fileName:
#   bpm, chartsData = main.open_ecg_from_id(str(val))
#   Q_wave_D3, Q_wave_V3, R_wave_V3, R_wave_V6 = return_calculations(chartsData)
#   heart_rate.append(bpm)
#   avg_width_Q_wave_D3.append(Q_wave_D3)
#   avg_width_Q_wave_V3.append(Q_wave_V3)
#   avg_width_R_wave_V3.append(R_wave_V3)
#   avg_width_R_wave_V6.append(R_wave_V6)

